In [9]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.memory import ConversationBufferMemory

llm = ChatOpenAI(
    temperature = 0.1,
    model = "gpt-4o-mini",
)

filename = 'document.txt'

loader = UnstructuredFileLoader(f'../{filename}')
cache_dir = LocalFileStore('../cache/')
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator = '\n\n',
    chunk_size = 5000,
    chunk_overlap = 100,
)


docs = loader.load_and_split(text_splitter = splitter)
embeddings = OpenAIEmbeddings()
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)
vectorstore = FAISS.from_documents(docs, cached_embeddings)
retriever = vectorstore.as_retriever()
memory = ConversationBufferMemory(
    llm = llm,
    return_messages = True,
    memory_key = 'history',
)

def load_memory (_):
    return memory.load_memory_variables({})["history"]

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system","""
            당신은 문서를 받고, 그 문서를 파악하여 대답해주는 AI에요.
            만약 해당하는 정보를 문서에서 찾지 못하는 경우엔 어떠한 경우에도 정보를 생성하지 말고 '모르겠습니다.' 라고 답변해주세요.
            무슨 일이 있어도 한국어로만 대답해주세요.
            {context}
            """,
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough(),
        "history": load_memory(),
    }
    | prompt
    | llm
)






FileNotFoundError: [Errno 2] No such file or directory: '../document.txt'